In [1]:
# import library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
ff_matches = pd.read_excel('Fifa_world_cup_raw3.xlsx', sheet_name='All Matches')
# ff_defails = pd.read_excel(file_path, sheet_name='Match Details')

In [3]:
# data information
ff_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 926 entries, 0 to 925
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        926 non-null    int64 
 1   Year              926 non-null    int64 
 2   Stage             926 non-null    object
 3   Date              926 non-null    object
 4   Time              926 non-null    object
 5   Stadium           926 non-null    object
 6   Location          926 non-null    object
 7   Home Team         926 non-null    object
 8   HT Goals          926 non-null    int64 
 9   AT Goals          926 non-null    object
 10  Away Team         926 non-null    object
 11  Match Attendance  926 non-null    object
 12  MatchID           926 non-null    int64 
 13  Refer             925 non-null    object
dtypes: int64(4), object(10)
memory usage: 101.4+ KB


In [4]:
# drop the first columns since no use
ff_matches = ff_matches.drop(columns=ff_matches.columns[0])
# ff_defails = ff_defails.drop(columns=ff_defails.columns[0])

In [5]:
# convert Date to datetime type
ff_matches['Date'] = pd.to_datetime(ff_matches['Date'])

In [6]:
# create column Winstats, New HT Goals, New AT Goals
ff_matches['Winstats'] = 0
ff_matches['New HT Goals'] = ff_matches['HT Goals'].copy()
ff_matches['New AT Goals'] = ff_matches['AT Goals'].copy()

In [7]:
# replace non-numeric chatacters from New HT Goals, New AT Goals
ff_matches[['New AT Goals','New HT Goals']] = ff_matches[['New AT Goals','New HT Goals']].replace(to_replace='\D+', value='', regex=True)

In [8]:
# clean up leading and ending spaces
ff_matches = ff_matches.replace(to_replace=r'^\\s+', value='', regex=True)
ff_matches = ff_matches.replace(to_replace=r'\\s+$', value='', regex=True)

In [9]:
# astype the 2 columns above to 'int64' dtype
ff_matches[['New HT Goals', 'New AT Goals']] = ff_matches[['New HT Goals', 'New AT Goals']].astype('int64')

In [10]:
# set Winstats = 1 if values in New HT Goals larger than New AT Goals
ff_matches['Winstats'][ff_matches['New HT Goals']-ff_matches['New AT Goals']>0] = 1

<ipython-input-10-ed723b5d971d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff_matches['Winstats'][ff_matches['New HT Goals']-ff_matches['New AT Goals']>0] = 1


In [11]:
# set Winstats = -1 if values in New HT Goals smaller than New AT Goals
ff_matches['Winstats'][ff_matches['New HT Goals']-ff_matches['New AT Goals']< 0] = -1

<ipython-input-11-fc3a89d04875>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff_matches['Winstats'][ff_matches['New HT Goals']-ff_matches['New AT Goals']< 0] = -1


In [12]:
# retrieve attended country
countries_list = set(pd.concat([ff_matches['Home Team'], ff_matches['Away Team']], axis=0))

In [13]:
# Generate winrate of countries_list
win_dict = {}
for country in countries_list:    
    for i, name in enumerate(ff_matches['Home Team']):
        if country == name:
            if country not in win_dict:
                win_dict[country] = {'win':0, 'draw': 0, 'lose': 0}
            if ff_matches.loc[i, 'Winstats'] == 1:
                win_dict[country]['win'] += 1
            elif ff_matches.loc[i, 'Winstats'] == -1:
                win_dict[country]['lose'] += 1
            elif ff_matches.loc[i, 'Winstats'] == 0:
                win_dict[country]['draw'] += 1

In [14]:
# create df_winrate_by_country
df_win_of_country = pd.DataFrame(win_dict)